In [14]:
import geoutils.utils.general_utils as gut
import geoutils.plotting.plots as cplt
import geoutils.utils.time_utils as tu
import geoutils.tsa.event_synchronization as es
import geoutils.geodata.base_dataset as bds
import climnet.datasets.evs_dataset as cds
import numpy as np
import climnet.network.clim_networkx as nx

from importlib import reload


In [7]:
vname = 'pr'
name = 'mswep'
start_month = 'Jun'
end_month = 'Sep'
grid_type = "fekete"
grid_step = 2.5
q_ee = 0.9

output_dir = '/home/strnad/data/climnet/outputs/'
plot_dir = '/home/strnad/data/climnet/plots/'
data_dir = "/home/strnad/data/climate_data/"
output_folder = 'summer_monsoon'

name_prefix = f"{name}_{grid_type}_{grid_step}_{q_ee}"

dataset_file = output_dir + \
    f"/{output_folder}/{name_prefix}_evs_ds.nc"

reload(cds)
ds = cds.EvsDataset(
    load_nc=dataset_file,
    rrevs=False,
)

Loading Data...
Load Dataset: /home/strnad/data/climnet/outputs//summer_monsoon/mswep_fekete_2.5_0.9_evs_ds.nc


Variables in dataset: ['pr', 'evs']
Set variable name to evs!
WARNING! No mask initialized!
Evs datat is stored in dataset.
Init spatial evs-mask for EVS data of shape: (15339, 5981)
... Finished Initialization EVS-spatial mask
Init the point-idx dictionaries


## Null model of the network

In [10]:
ds_jjas = tu.get_month_range_data(ds.ds, 'Jun', 'Sep')
num_time_steps = len(ds_jjas.time)

Select data from Jun - Sep!


In [ ]:
taumax = 10
n_pmts = 1000
weighted = True
min_evs = 3
# Init q-null models values
q_range = np.array([0.25, 0.5, 0.75, 0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999])

max_num_events = int(np.ceil(num_time_steps*(1-q_ee)))
null_model_folder = f'~/climnet/monsoon/climnet/network/es_files/null_model/'
null_model_file = f'null_model_ts_{num_time_steps}_taumax_{taumax}_npmts_{n_pmts}_q_{q_ee}_directed.npy'
nmfp = null_model_folder + null_model_file
# %%
reload(es)

q_dict = es.null_model_distribution(length_time_series=num_time_steps,
                                    max_num_events=max_num_events,
                                    num_permutations=n_pmts,
                                    taumax=taumax,
                                    min_num_events=1,
                                    q=q_range,
                                    savepath=nmfp)

E_matrix_folder = (
    f"{output_folder}/{name_prefix}_{q_ee}_{min_evs}/"
)
null_model_file = f'null_model_ts_{num_time_steps}_taumax_{taumax}_npmts_{n_pmts}_q_{q_ee}_directed.npy'

## Create the network

In [16]:
networkfile = output_dir + f"{output_folder}/{name_prefix}_ES_net.npz"

E_matrix_folder = (
    f"{output_folder}/{name_prefix}_{q_ee}_{min_evs}/"
)
null_model_file = f'null_model_ts_{num_time_steps}_taumax_{taumax}_npmts_{n_pmts}_q_{q_ee}_directed.npy'

q_sig = 0.95
Net = nx.Clim_NetworkX(dataset=ds,
                       taumax=taumax,
                       weighted=weighted,)
gut.myprint(f"Use q = {q_sig}")
Net.create(
    method='es',
    null_model_file=null_model_file,
    E_matrix_folder=E_matrix_folder,
    q_sig=q_sig,
    num_cpus=1,
)


Use q = 0.95
Create Network based on method es
Load null model: /home/strnad/climnet/monsoon/climnet/network/es_files//null_model//null_model_ts_5124_taumax_10_npmts_1000_q_0.9_directed.npy
Not running with SLURM job arrays, but with manual id:  0
Computed Event Series Matrix
Store E-matrix as file E_matrix_evs_q_0.9_min_num_events_3_taumax_10_jobid_0.npy!
JobID 0: Start comparing all time series with taumax=10!
Start computing event synchronization for event data from 0 to 5250!
Use 1 CPUs in parallel!


  0%|          | 0/5249 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
nx_path_file = output_dir + \
    f"{output_folder}/{name_prefix}_{q_sig}_ES_nx.gml.gz"
Net.save(nx_path_file)